In [28]:
flatten = lambda t: [item for sublist in t for item in sublist]
sentences = [
    "s the green bottle leaked .",
    "s the suppliers bottle water .",
    "s green water suppliers bottle .",
]

sentences = [sentence.split() for sentence in sentences]
tags = [
    "S DTO AJO NN1 VVD PUN",
    "S DTO NN2 VVB NN1 PUN",
    "S AJO NN1 NN2 VVB PUN",
]
tags = [tag.split() for tag in tags]
flat_tags = flatten(tags)
flat_sentences = flatten(sentences)
voc = set(flatten(tags))

In [77]:
to_bigram = lambda x: [(t_1, t_2) for t_1, t_2 in zip(x[1:], x[:-1] )]
bigram_tags = [to_bigram(tag) for tag in tags]

print(bigram_tags)



[[('DTO', 'S'), ('AJO', 'DTO'), ('NN1', 'AJO'), ('VVD', 'NN1'), ('PUN', 'VVD')], [('DTO', 'S'), ('NN2', 'DTO'), ('VVB', 'NN2'), ('NN1', 'VVB'), ('PUN', 'NN1')], [('AJO', 'S'), ('NN1', 'AJO'), ('NN2', 'NN1'), ('VVB', 'NN2'), ('PUN', 'VVB')]]


In [78]:
### Get the counts
from collections import Counter, defaultdict
flat_bigram_tags = []
for tag in bigram_tags:
    flat_bigram_tags += tag
bigram_count = Counter(flat_bigram_tags)

In [124]:
### Get the probabilities:
bigram_probs = defaultdict(float)
total_counts = defaultdict(float)
for item, count in bigram_count.items():
    total_counts[item[1]] += count

for item, count in bigram_count.items():
    bigram_probs[item] = count / total_counts[item[1]]
print(bigram_probs)


defaultdict(<class 'float'>, {('DTO', 'S'): 0.6666666666666666, ('AJO', 'DTO'): 0.5, ('NN1', 'AJO'): 1.0, ('VVD', 'NN1'): 0.3333333333333333, ('PUN', 'VVD'): 1.0, ('NN2', 'DTO'): 0.5, ('VVB', 'NN2'): 1.0, ('NN1', 'VVB'): 0.5, ('PUN', 'NN1'): 0.3333333333333333, ('AJO', 'S'): 0.3333333333333333, ('NN2', 'NN1'): 0.3333333333333333, ('PUN', 'VVB'): 0.5})


In [125]:
### Get the word probabilites

word_tag_pairs = [(word, tag) for word, tag in zip(flat_sentences, flat_tags)]
print(word_tag_pairs)

[('s', 'S'), ('the', 'DTO'), ('green', 'AJO'), ('bottle', 'NN1'), ('leaked', 'VVD'), ('.', 'PUN'), ('s', 'S'), ('the', 'DTO'), ('suppliers', 'NN2'), ('bottle', 'VVB'), ('water', 'NN1'), ('.', 'PUN'), ('s', 'S'), ('green', 'AJO'), ('water', 'NN1'), ('suppliers', 'NN2'), ('bottle', 'VVB'), ('.', 'PUN')]


In [126]:
word_tag_count = Counter(word_tag_pairs)

In [127]:
### Get the probabilities:
word_tag_probs = defaultdict(float)

total_counts = defaultdict(float)
for item, count in word_tag_count.items():
    total_counts[item[1]] += count

for (word, tag) in word_tag_pairs:
    word_tag_probs[(word, tag)] = word_tag_count[(word, tag)] / total_counts[tag]
print(word_tag_probs)

defaultdict(<class 'float'>, {('s', 'S'): 1.0, ('the', 'DTO'): 1.0, ('green', 'AJO'): 1.0, ('bottle', 'NN1'): 0.3333333333333333, ('leaked', 'VVD'): 1.0, ('.', 'PUN'): 1.0, ('suppliers', 'NN2'): 1.0, ('bottle', 'VVB'): 1.0, ('water', 'NN1'): 0.6666666666666666})


In [130]:
### Lastly calculate the probabilities of a sentence
def calc_prob(s, t, bigram_probs, word_tag_probs):
    word_tag_pairs = [(word, tag) for word, tag in zip(s, t)]
    bigram = to_bigram(t)
    total = 1
    sentence= ""
    written_out = ""
    for pair, bigram in zip(word_tag_pairs, bigram):
        sentence += " P({}|{}) P({}|{})".format(pair[0], pair[1], bigram[0], bigram[1])
        written_out += "{:.2f} \cdot {:.2f} \cdot ".format(word_tag_probs[pair], bigram_probs[bigram] )

        total *= word_tag_probs[pair] * bigram_probs[bigram]
        
    final_sentence = "P({}| {}) \n  = $$ \cdot$$   {}  \n  = $$ 3 \cdot  {} $$  \n \n $$ \\approx {:.2f}$$".format(" ".join(t), " ".join(s), sentence, written_out, total)
    print(final_sentence)
    return total
    

In [131]:
result = [calc_prob(s, t,  bigram_probs, word_tag_probs ) for  s, t in zip(sentences, tags)]

print(  result)

P(S DTO AJO NN1 VVD PUN| s the green bottle leaked .) 
  = $$ \cdot$$    P(s|S) P(DTO|S) P(the|DTO) P(AJO|DTO) P(green|AJO) P(NN1|AJO) P(bottle|NN1) P(VVD|NN1) P(leaked|VVD) P(PUN|VVD)  
  = $$ 3 \cdot  1.00 \cdot 0.67 \cdot 1.00 \cdot 0.50 \cdot 1.00 \cdot 1.00 \cdot 0.33 \cdot 0.33 \cdot 1.00 \cdot 1.00 \cdot  $$  
 
 $$ \approx 0.04$$
P(S DTO NN2 VVB NN1 PUN| s the suppliers bottle water .) 
  = $$ \cdot$$    P(s|S) P(DTO|S) P(the|DTO) P(NN2|DTO) P(suppliers|NN2) P(VVB|NN2) P(bottle|VVB) P(NN1|VVB) P(water|NN1) P(PUN|NN1)  
  = $$ 3 \cdot  1.00 \cdot 0.67 \cdot 1.00 \cdot 0.50 \cdot 1.00 \cdot 1.00 \cdot 1.00 \cdot 0.50 \cdot 0.67 \cdot 0.33 \cdot  $$  
 
 $$ \approx 0.04$$
P(S AJO NN1 NN2 VVB PUN| s green water suppliers bottle .) 
  = $$ \cdot$$    P(s|S) P(AJO|S) P(green|AJO) P(NN1|AJO) P(water|NN1) P(NN2|NN1) P(suppliers|NN2) P(VVB|NN2) P(bottle|VVB) P(PUN|VVB)  
  = $$ 3 \cdot  1.00 \cdot 0.33 \cdot 1.00 \cdot 1.00 \cdot 0.67 \cdot 0.33 \cdot 1.00 \cdot 1.00 \cdot 1.00 \cdot 0.